In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install scikit-optimize

     |████████████████████████████████| 102kB 3.1MB/s 


In [4]:
import pandas as pd
import sys
sys.path.insert(0,'/content/drive/My Drive/Datasets_SVR/Library')
from subsamplesvr import svr_subsample_bayes_train
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.svm import SVR
import numpy as np
import time
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV

In [5]:
data=pd.read_csv("/content/drive/My Drive/Datasets_SVR/Data/PRSA_data_2010.1.1-2014.12.31 (1).csv")

In [6]:
data.describe()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir
count,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,41757.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000
mean,21912.500000,2012.000000,6.523549,15.727820,11.500000,98.613215,1.817246,12.448521,1016.447654,23.889140,0.052734,0.194916
std,12651.043435,1.413842,3.448572,8.799425,6.922266,92.050387,14.433440,12.198613,10.268698,50.010635,0.760375,1.415867
min,1.000000,2010.000000,1.000000,1.000000,0.000000,0.000000,-40.000000,-19.000000,991.000000,0.450000,0.000000,0.000000
25%,10956.750000,2011.000000,4.000000,8.000000,5.750000,29.000000,-10.000000,2.000000,1008.000000,1.790000,0.000000,0.000000
50%,21912.500000,2012.000000,7.000000,16.000000,11.500000,72.000000,2.000000,14.000000,1016.000000,5.370000,0.000000,0.000000
75%,32868.250000,2013.000000,10.000000,23.000000,17.250000,137.000000,15.000000,23.000000,1025.000000,21.910000,0.000000,0.000000
max,43824.000000,2014.000000,12.000000,31.000000,23.000000,994.000000,28.000000,42.000000,1046.000000,585.600000,27.000000,36.000000


In [7]:
print(data.shape)

(43824, 13)


In [8]:
data.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [9]:
data.isna().sum()

No          0
year        0
month       0
day         0
hour        0
pm2.5    2067
DEWP        0
TEMP        0
PRES        0
cbwd        0
Iws         0
Is          0
Ir          0
dtype: int64

In [10]:
data=data.drop(data.columns[[0]], axis='columns')

In [11]:
data

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43819,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [12]:
data=data.dropna()

In [13]:
data.isna().sum()

year     0
month    0
day      0
hour     0
pm2.5    0
DEWP     0
TEMP     0
PRES     0
cbwd     0
Iws      0
Is       0
Ir       0
dtype: int64

In [14]:
data.shape

(41757, 12)

In [15]:
data.reset_index(drop=True,inplace=True)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41757 entries, 0 to 41756
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    41757 non-null  int64  
 1   month   41757 non-null  int64  
 2   day     41757 non-null  int64  
 3   hour    41757 non-null  int64  
 4   pm2.5   41757 non-null  float64
 5   DEWP    41757 non-null  int64  
 6   TEMP    41757 non-null  float64
 7   PRES    41757 non-null  float64
 8   cbwd    41757 non-null  object 
 9   Iws     41757 non-null  float64
 10  Is      41757 non-null  int64  
 11  Ir      41757 non-null  int64  
dtypes: float64(4), int64(7), object(1)
memory usage: 3.8+ MB


In [17]:
data.cbwd.value_counts()

SE    14573
NW    13484
cv     8944
NE     4756
Name: cbwd, dtype: int64

In [18]:
data['cbwd'].replace(to_replace=['SE'],value=0,inplace=True)
data['cbwd'].replace(to_replace=['NW'],value=1,inplace=True)
data['cbwd'].replace(to_replace=['cv'],value=2,inplace=True)
data['cbwd'].replace(to_replace=['NE'],value=3,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [19]:
data.cbwd.value_counts()

0    14573
1    13484
2     8944
3     4756
Name: cbwd, dtype: int64

In [20]:
data

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,2010,1,2,0,129.0,-16,-4.0,1020.0,0,1.79,0,0
1,2010,1,2,1,148.0,-15,-4.0,1020.0,0,2.68,0,0
2,2010,1,2,2,159.0,-11,-5.0,1021.0,0,3.57,0,0
3,2010,1,2,3,181.0,-7,-5.0,1022.0,0,5.36,1,0
4,2010,1,2,4,138.0,-7,-5.0,1022.0,0,6.25,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
41752,2014,12,31,19,8.0,-23,-2.0,1034.0,1,231.97,0,0
41753,2014,12,31,20,10.0,-22,-3.0,1034.0,1,237.78,0,0
41754,2014,12,31,21,10.0,-22,-3.0,1034.0,1,242.70,0,0
41755,2014,12,31,22,8.0,-22,-4.0,1034.0,1,246.72,0,0


In [21]:
data['year'] = data['year'].astype(str)
data['month'] = data['month'].astype(str)
data['day'] = data['day'].astype(str)
data['hour'] = data['hour'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41757 entries, 0 to 41756
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    41757 non-null  object 
 1   month   41757 non-null  object 
 2   day     41757 non-null  object 
 3   hour    41757 non-null  object 
 4   pm2.5   41757 non-null  float64
 5   DEWP    41757 non-null  int64  
 6   TEMP    41757 non-null  float64
 7   PRES    41757 non-null  float64
 8   cbwd    41757 non-null  int64  
 9   Iws     41757 non-null  float64
 10  Is      41757 non-null  int64  
 11  Ir      41757 non-null  int64  
dtypes: float64(4), int64(4), object(4)
memory usage: 3.8+ MB


In [23]:
data["Date"]=data['year']+"/"+data['month']+"/"+data['day']+" "+data['hour']+":00:00"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
data

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,Date
0,2010,1,2,0,129.0,-16,-4.0,1020.0,0,1.79,0,0,2010/1/2 0:00:00
1,2010,1,2,1,148.0,-15,-4.0,1020.0,0,2.68,0,0,2010/1/2 1:00:00
2,2010,1,2,2,159.0,-11,-5.0,1021.0,0,3.57,0,0,2010/1/2 2:00:00
3,2010,1,2,3,181.0,-7,-5.0,1022.0,0,5.36,1,0,2010/1/2 3:00:00
4,2010,1,2,4,138.0,-7,-5.0,1022.0,0,6.25,2,0,2010/1/2 4:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41752,2014,12,31,19,8.0,-23,-2.0,1034.0,1,231.97,0,0,2014/12/31 19:00:00
41753,2014,12,31,20,10.0,-22,-3.0,1034.0,1,237.78,0,0,2014/12/31 20:00:00
41754,2014,12,31,21,10.0,-22,-3.0,1034.0,1,242.70,0,0,2014/12/31 21:00:00
41755,2014,12,31,22,8.0,-22,-4.0,1034.0,1,246.72,0,0,2014/12/31 22:00:00


In [25]:
data['Date'] = pd.to_datetime(data['Date'], format='%Y/%m/%d %H:%M:%S')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41757 entries, 0 to 41756
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   year    41757 non-null  object        
 1   month   41757 non-null  object        
 2   day     41757 non-null  object        
 3   hour    41757 non-null  object        
 4   pm2.5   41757 non-null  float64       
 5   DEWP    41757 non-null  int64         
 6   TEMP    41757 non-null  float64       
 7   PRES    41757 non-null  float64       
 8   cbwd    41757 non-null  int64         
 9   Iws     41757 non-null  float64       
 10  Is      41757 non-null  int64         
 11  Ir      41757 non-null  int64         
 12  Date    41757 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(4), object(4)
memory usage: 4.1+ MB


In [27]:
data.drop(data.columns[[0,1,2,3]],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
data

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,Date
0,129.0,-16,-4.0,1020.0,0,1.79,0,0,2010-01-02 00:00:00
1,148.0,-15,-4.0,1020.0,0,2.68,0,0,2010-01-02 01:00:00
2,159.0,-11,-5.0,1021.0,0,3.57,0,0,2010-01-02 02:00:00
3,181.0,-7,-5.0,1022.0,0,5.36,1,0,2010-01-02 03:00:00
4,138.0,-7,-5.0,1022.0,0,6.25,2,0,2010-01-02 04:00:00
...,...,...,...,...,...,...,...,...,...
41752,8.0,-23,-2.0,1034.0,1,231.97,0,0,2014-12-31 19:00:00
41753,10.0,-22,-3.0,1034.0,1,237.78,0,0,2014-12-31 20:00:00
41754,10.0,-22,-3.0,1034.0,1,242.70,0,0,2014-12-31 21:00:00
41755,8.0,-22,-4.0,1034.0,1,246.72,0,0,2014-12-31 22:00:00


In [29]:
data.sort_values(by='Date',inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
data

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir,Date
0,129.0,-16,-4.0,1020.0,0,1.79,0,0,2010-01-02 00:00:00
1,148.0,-15,-4.0,1020.0,0,2.68,0,0,2010-01-02 01:00:00
2,159.0,-11,-5.0,1021.0,0,3.57,0,0,2010-01-02 02:00:00
3,181.0,-7,-5.0,1022.0,0,5.36,1,0,2010-01-02 03:00:00
4,138.0,-7,-5.0,1022.0,0,6.25,2,0,2010-01-02 04:00:00
...,...,...,...,...,...,...,...,...,...
41752,8.0,-23,-2.0,1034.0,1,231.97,0,0,2014-12-31 19:00:00
41753,10.0,-22,-3.0,1034.0,1,237.78,0,0,2014-12-31 20:00:00
41754,10.0,-22,-3.0,1034.0,1,242.70,0,0,2014-12-31 21:00:00
41755,8.0,-22,-4.0,1034.0,1,246.72,0,0,2014-12-31 22:00:00


In [31]:
data['Date'].min()

Timestamp('2010-01-02 00:00:00')

In [32]:
data['Date'].max()

Timestamp('2014-12-31 23:00:00')

In [33]:
data.drop('Date',axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [34]:
data

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,129.0,-16,-4.0,1020.0,0,1.79,0,0
1,148.0,-15,-4.0,1020.0,0,2.68,0,0
2,159.0,-11,-5.0,1021.0,0,3.57,0,0
3,181.0,-7,-5.0,1022.0,0,5.36,1,0
4,138.0,-7,-5.0,1022.0,0,6.25,2,0
...,...,...,...,...,...,...,...,...
41752,8.0,-23,-2.0,1034.0,1,231.97,0,0
41753,10.0,-22,-3.0,1034.0,1,237.78,0,0
41754,10.0,-22,-3.0,1034.0,1,242.70,0,0
41755,8.0,-22,-4.0,1034.0,1,246.72,0,0


In [35]:
data.describe()

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
count,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000,41757.000000
mean,98.613215,1.750174,12.401561,1016.442896,1.092990,23.866747,0.055344,0.194866
std,92.050387,14.433658,12.175215,10.300733,1.005063,49.617495,0.778875,1.418165
min,0.000000,-40.000000,-19.000000,991.000000,0.000000,0.450000,0.000000,0.000000
25%,29.000000,-10.000000,2.000000,1008.000000,0.000000,1.790000,0.000000,0.000000
50%,72.000000,2.000000,14.000000,1016.000000,1.000000,5.370000,0.000000,0.000000
75%,137.000000,15.000000,23.000000,1025.000000,2.000000,21.910000,0.000000,0.000000
max,994.000000,28.000000,42.000000,1046.000000,3.000000,565.490000,27.000000,36.000000


In [36]:
train_size=int(data.shape[0]*0.8)

In [37]:
train_size

33405

In [38]:
x_train=data.iloc[:train_size]

In [39]:
x_train

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,129.0,-16,-4.0,1020.0,0,1.79,0,0
1,148.0,-15,-4.0,1020.0,0,2.68,0,0
2,159.0,-11,-5.0,1021.0,0,3.57,0,0
3,181.0,-7,-5.0,1022.0,0,5.36,1,0
4,138.0,-7,-5.0,1022.0,0,6.25,2,0
...,...,...,...,...,...,...,...,...
33400,177.0,-18,-7.0,1031.0,3,1.79,0,0
33401,287.0,-17,-7.0,1032.0,0,0.89,0,0
33402,299.0,-19,-8.0,1032.0,3,1.79,0,0
33403,250.0,-18,-7.0,1033.0,1,1.79,0,0


In [40]:
x_test=data.iloc[train_size:]

In [41]:
x_test

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
33405,124.0,-20,-9.0,1033.0,1,6.71,0,0
33406,123.0,-19,-8.0,1034.0,1,11.63,0,0
33407,49.0,-21,-10.0,1034.0,1,15.65,0,0
33408,36.0,-19,-9.0,1034.0,2,0.89,0,0
33409,59.0,-20,-9.0,1034.0,2,1.78,0,0
...,...,...,...,...,...,...,...,...
41752,8.0,-23,-2.0,1034.0,1,231.97,0,0
41753,10.0,-22,-3.0,1034.0,1,237.78,0,0
41754,10.0,-22,-3.0,1034.0,1,242.70,0,0
41755,8.0,-22,-4.0,1034.0,1,246.72,0,0


In [42]:
from sklearn.utils import shuffle

In [43]:
x_train=shuffle(x_train)

In [44]:
x_test=shuffle(x_test)

In [45]:
x_train

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
20541,107.0,24,28.0,1002.0,0,6.71,0,0
7428,100.0,-8,-1.0,1013.0,1,3.13,0,0
22415,259.0,13,13.0,1012.0,1,3.13,0,0
2527,16.0,2,13.0,1021.0,1,5.81,0,0
29766,78.0,22,32.0,1003.0,0,3.58,0,0
...,...,...,...,...,...,...,...,...
15529,281.0,0,0.0,1031.0,3,0.89,3,0
1172,84.0,-10,4.0,1016.0,0,45.60,0,0
9868,20.0,-18,2.0,1026.0,1,114.89,0,0
27954,17.0,-11,31.0,1006.0,1,128.29,0,0


In [46]:
x_test

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
34536,154.0,-8,3.0,1025.0,3,1.79,0,0
40934,250.0,0,7.0,1015.0,0,1.79,0,0
34563,295.0,-7,3.0,1021.0,0,1.79,0,0
37943,16.0,13,32.0,1002.0,1,3.13,0,0
35081,320.0,0,7.0,1011.0,1,1.78,0,0
...,...,...,...,...,...,...,...,...
39666,81.0,14,15.0,1017.0,0,9.84,0,5
34596,9.0,-25,0.0,1030.0,1,99.69,0,0
40837,31.0,-11,11.0,1028.0,1,26.83,0,0
39525,49.0,12,14.0,1017.0,3,1.78,0,0


In [47]:
col_names=list(data.columns[data.columns!="pm2.5"])

In [48]:
col_names

['DEWP', 'TEMP', 'PRES', 'cbwd', 'Iws', 'Is', 'Ir']

In [49]:
scaler = StandardScaler()
scaled_data=scaler.fit_transform(x_train.loc[:,x_train.columns!="pm2.5"])

In [50]:
x_train[col_names]=scaled_data

In [51]:
scaler = StandardScaler()
scaled_data=scaler.fit_transform(x_test.loc[:,x_test.columns!="pm2.5"])

In [52]:
x_test[col_names]=scaled_data

In [53]:
x_train

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
20541,107.0,1.545965,1.304504,-1.386405,-1.094501,-0.358287,-0.076390,-0.13993
7428,100.0,-0.652771,-1.050353,-0.334214,-0.092972,-0.428785,-0.076390,-0.13993
22415,259.0,0.790149,0.086474,-0.429868,-0.092972,-0.428785,-0.076390,-0.13993
2527,16.0,0.034334,0.086474,0.431016,-0.092972,-0.376010,-0.076390,-0.13993
29766,78.0,1.408544,1.629311,-1.290751,-1.094501,-0.419924,-0.076390,-0.13993
...,...,...,...,...,...,...,...,...
15529,281.0,-0.103087,-0.969151,1.387553,1.910084,-0.472896,3.707812,-0.13993
1172,84.0,-0.790192,-0.644343,-0.047253,-1.094501,0.407544,-0.076390,-0.13993
9868,20.0,-1.339876,-0.806747,0.909285,-0.092972,1.772020,-0.076390,-0.13993
27954,17.0,-0.858903,1.548110,-1.003790,-0.092972,2.035896,-0.076390,-0.13993


In [54]:
x_test

,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
34536,154.0,-0.773515,-0.987542,0.907132,1.849161,-0.403620,-0.047879,-0.130496
40934,250.0,-0.197850,-0.636957,-0.128230,-1.060811,-0.403620,-0.047879,-0.130496
34563,295.0,-0.701557,-0.987542,0.492987,-1.060811,-0.403620,-0.047879,-0.130496
37943,16.0,0.737605,1.554199,-1.474201,-0.090820,-0.373451,-0.047879,-0.130496
35081,320.0,-0.197850,-0.636957,-0.542375,-0.090820,-0.403846,-0.047879,-0.130496
...,...,...,...,...,...,...,...,...
39666,81.0,0.809563,0.064213,0.078842,-1.060811,-0.222376,-0.047879,4.616450
34596,9.0,-1.996803,-1.250481,1.424813,-0.090820,1.800589,-0.047879,-0.130496
40837,31.0,-0.989389,-0.286372,1.217741,-0.090820,0.160153,-0.047879,-0.130496
39525,49.0,0.665647,-0.023433,0.078842,1.849161,-0.403846,-0.047879,-0.130496


## Baseline for comparison...

In [ ]:
import math
#('C', 1000.0), ('degree', 2), ('epsilon', 11.654346197659505), ('kernel', 'rbf')]
start = time.time()
base_rbf=SVR(kernel='rbf',C=1000.0,epsilon=11.654346197659505).fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred_rbf = base_rbf.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred_rbf))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred_rbf)))
print("SCORE: ",base_rbf.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

MAE:  50.937021334549875
RMSE:  78.58830238297938
SCORE:  0.30517847354886485
total time: 88.50055646896362


In [ ]:
#('C', 21.45826038255209), ('degree', 5), ('epsilon', 56.12491210516413), ('kernel', 'linear')
start = time.time()
base_linear=SVR(kernel='linear',C=21.45826038255209,epsilon=56.12491210516413).fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred_linear = base_linear.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred_linear))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred_linear)))
print("SCORE: ",base_linear.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

MAE:  58.987144188437995
RMSE:  84.20233086087292
SCORE:  0.20236229126741312
total time: 31.46774959564209


In [ ]:
#('C', 122.98194954592115), ('degree', 3), ('epsilon', 13.471309143419143), ('kernel', 'poly')]
start = time.time()
base_poly=SVR(kernel='poly',C=122.98194954592115,epsilon=13.471309143419143,degree=3).fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred_poly = base_poly.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred_poly))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred_poly)))
print("SCORE: ",base_poly.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

MAE:  58.73260499322982
RMSE:  88.34935206326742
SCORE:  0.12185913552770343
total time: 1069.0881416797638


In [ ]:
#('C', 0.1016883125670228), ('degree', 3), ('epsilon', 48.47080968794483), ('kernel', 'sigmoid')]
start = time.time()
base_sig=SVR(kernel='sigmoid',C=0.1016883125670228,degree=3,epsilon=48.47080968794483).fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred_sig = base_sig.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred_sig))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred_sig)))
print("SCORE: ",base_sig.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

MAE:  62.102107942922224
RMSE:  90.19843232630117
SCORE:  0.08471693090003785
total time: 50.23502779006958


## SVR subsmple with Bayesian Optimization

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"pm2.5","rbf",5)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.3s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.9s finished


++++++ TRAIN TIME (1) 138.23579597473145 +++++++++++
####### BEST MODEL PARAMS STEP 1 #####
OrderedDict([('C', 1000.0), ('degree', 2), ('epsilon', 11.654346197659505), ('kernel', 'rbf')])
Coefficient of determination R^2 of the prediction: 

0.2985580739837025
MAE:  52.26662841043599
RMSE:  78.96181735425432
DIFF with previous:  -0.0701980035787291
####### BEST MODEL PARAMS STEP 1 #####
!!!! TOTAL ELAPSED TIME: 225.34913039207458 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=50, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['rbf']},
              verbose=3)

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"pm2.5","linear",5)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.3s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    6.2s finished


++++++ TRAIN TIME (1) 138.31502270698547 +++++++++++
####### BEST MODEL PARAMS STEP 1 #####
OrderedDict([('C', 21.45826038255209), ('degree', 5), ('epsilon', 56.12491210516413), ('kernel', 'linear')])
Coefficient of determination R^2 of the prediction: 

0.20589811917608902
MAE:  58.79028373417264
RMSE:  84.01549438997606
DIFF with previous:  -0.030420623812565606
####### BEST MODEL PARAMS STEP 1 #####
!!!! TOTAL ELAPSED TIME: 213.47943353652954 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=50, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['linear']},
              verbose=3)

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"pm2.5","poly",5)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    1.3s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.7s finished


++++++ TRAIN TIME (1) 1361.5323355197906 +++++++++++
####### BEST MODEL PARAMS STEP 1 #####
OrderedDict([('C', 122.98194954592115), ('degree', 3), ('epsilon', 13.471309143419143), ('kernel', 'poly')])
Coefficient of determination R^2 of the prediction: 

-0.063992126974034
MAE:  59.81085877988717
RMSE:  97.25019531389444
DIFF with previous:  -0.023262438606298952
####### BEST MODEL PARAMS STEP 1 #####
!!!! TOTAL ELAPSED TIME: 1463.9338593482971 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=50, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['poly']},
              verbose=3)

In [ ]:
svr_subsample_bayes_train(x_train,x_test,0.01,0.1,"pm2.5","sigmoid",5)

####### BAYESIAN_OPT MODE #######
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    0.0s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   15.4s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   13.9s finished


++++++ TRAIN TIME (ITER) 152.52094721794128 +++++++++++
####### BEST MODEL PARAMS ITER 0 #####
OrderedDict([('C', 0.1016883125670228), ('degree', 3), ('epsilon', 48.47080968794483), ('kernel', 'sigmoid')])
Coefficient of determination R^2 of the prediction: 

0.05952312528750692
MAE:  62.963133447904085
RMSE:  91.43139281467292
DIFF with previous:  -0.0821737429241326
####### BEST MODEL PARAMS ITER 0 #####
!!!! TOTAL ELAPSED TIME: 392.36113262176514 !!!!


BayesSearchCV(cv=3, error_score='raise',
              estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                            epsilon=0.1, gamma='scale', kernel='rbf',
                            max_iter=-1, shrinking=True, tol=0.001,
                            verbose=False),
              fit_params=None, iid=True, n_iter=10, n_jobs=3, n_points=1,
              optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
              refit=True, return_train_score=False, scoring=None,
              search_spaces={'C': (0.01, 1000.0, 'log-uniform'),
                             'degree': (2, 5),
                             'epsilon': (0.01, 100.0, 'log-uniform'),
                             'kernel': ['sigmoid']},
              verbose=3)

## Comparison without the subsample algorithm

In [ ]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['rbf']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    verbose=3
)

Sv.fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   55.5s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   51.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   36.3s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.1min finished


MAE:  52.585505580903934
RMSE:  77.79346591019075
SCORE:  0.31916214968287704
total time: 2841.9822475910187


In [ ]:
Sv.best_params_

OrderedDict([('C', 1000.0),
             ('degree', 5),
             ('epsilon', 35.90121323345085),
             ('kernel', 'rbf')])

In [ ]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['linear']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    verbose=3
)

Sv.fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   35.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   34.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:   10.7s finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:    7.6s finished


MAE:  64.66077553869214
RMSE:  84.88254433848466
SCORE:  0.18942309072558972
total time: 3069.840247631073


In [ ]:
Sv.best_params_

OrderedDict([('C', 21.980550638376975),
             ('degree', 4),
             ('epsilon', 98.3188667203503),
             ('kernel', 'linear')])

In [ ]:
#didnt work 14 hours and 25 minutes
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['poly']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    verbose=3
)

Sv.fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

In [56]:
import math
start = time.time()
Sv = BayesSearchCV(
    SVR(gamma='scale'),
    {
        'C': (1e-2, 1e+3,'log-uniform'),
        'degree': (2, 5),
        'epsilon': (1e-2, 1e+2,'log-uniform'),  # integer valued parameter
        'kernel': ['sigmoid']  # categorical parameter
    },
    cv=3,
    n_jobs=3,
    verbose=3
)

Sv.fit(x_train.loc[:,x_train.columns!="pm2.5"], x_train.loc[:,"pm2.5"])
end = time.time()
pred = Sv.predict(x_test.loc[:,x_train.columns!="pm2.5"])
# mean_absolute_error(y_true, y_pred)
# mean_squared_error(y_true, y_pred)
print("MAE: ",mean_absolute_error(x_test.loc[:,"pm2.5"], pred))
print("RMSE: ",math.sqrt(mean_squared_error(x_test.loc[:,"pm2.5"], pred)))
print("SCORE: ",Sv.score(x_test.loc[:,x_train.columns!="pm2.5"], x_test.loc[:,"pm2.5"]))
print(str("total time: ")+ (str(end - start)))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.2min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.3min finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.2min finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   3 out of   3 | elapsed:  1.2min finished


MAE:  62.721186425387984
RMSE:  96.72522367667781
SCORE:  -0.052535942747858
total time: 3881.3814799785614


In [57]:
Sv.best_params_

OrderedDict([('C', 0.014419240497213591),
             ('degree', 2),
             ('epsilon', 0.36581707036316197),
             ('kernel', 'sigmoid')])